# Reference-free spot cell-type deconvolution with STdeconvolve

R script file available for download: https://raw.githubusercontent.com/10XGenomics/analysis_guides/main/reference_17p9_rep1_withastrocytes.v2.GitHub.R

# Install

In [ ]:
install.packages("remotes")
system("apt install libgsl-dev", intern=TRUE)

This can take ~10 min:

In [ ]:
remotes::install_github('JEFworks-Lab/STdeconvolve')

This can take ~20 min:

In [ ]:
remotes::install_version("Seurat", version = "4.3.0")

In [ ]:
install.packages("hdf5r")
install.packages("tidyverse")

# Load libraries

In [ ]:
library(Seurat)
library(STdeconvolve)

This might give errors. You can ignore them.

In [ ]:
library(tidyverse)

# Download input files

This will download input files needed for this tutorial.

In [ ]:
download.file("https://raw.githubusercontent.com/10XGenomics/analysis_guides/main/input_files/VisiumFFPE_Mouse_Brain_Transgenic_Age_17p9_Rep_1.h5", "VisiumFFPE_Mouse_Brain_Transgenic_Age_17p9_Rep_1.h5")
download.file("https://raw.githubusercontent.com/10XGenomics/analysis_guides/main/input_files/astro_markers.csv","astro_markers.csv")
download.file("https://raw.githubusercontent.com/10XGenomics/analysis_guides/main/input_files/spatial_cord_subset_17p9_rep1.csv", "spatial_cord_subset_17p9_rep1.csv")
download.file("https://raw.githubusercontent.com/10XGenomics/analysis_guides/main/input_files/optlDA.17p9_rep1_astrogenes.rds","optlDA.17p9_rep1_astrogenes.rds")
list.files(path = ".")

# Functions

In [ ]:
marker_gene_list<-function(topic=1,exp_value=2,Gexp){
  ## highly expressed in cell-type of interest
  highgexp <- names(which(Gexp[topic,] > exp_value))
  ## high log2(fold-change) compared to other deconvolved cell-types
  log2fc <- sort(log2(Gexp[topic,highgexp]/colMeans(Gexp[-topic,highgexp])), decreasing=TRUE)
  return(tibble(Gene=names(log2fc),log2fc=log2fc))
}

In [ ]:
plot_spatial<-function(plot_data=plot_data,suffix1='_prop.eps',suffix2='_prop_lim1.eps',y_max=1,dir=dir,i){
  p1<-ggplot(plot_data, aes(x, y,fill = prop)) +
    geom_point(shape=21,size=4) + 
    guides(size="none")+
    labs(title=str_c("topic ",i))+
    scale_fill_viridis_c() +
    theme_void()+
    coord_equal()
  
  p2<-ggplot(plot_data, aes(x, y,fill = prop)) +
    geom_point(shape=21,size=4) + 
    guides(size="none")+
    labs(title=str_c("topic ",i))+
    scale_fill_viridis_c(limits=c(0,y_max)) +
    theme_void()+
    coord_equal()
  ggsave(plot = p1, paste(dir,"topic_",i,suffix1, sep=''), 
         height=5, width=5, units='in', dpi=300)
  ggsave(plot= p2, paste(dir,"topic_",i,suffix2, sep=''), 
         height=5, width=5, units='in', dpi=300)
}

In [ ]:
run_me_results<-function(opt,
                         dir ){
  optLDA <- optimalModel(models = ldas, opt = opt)
  results <- getBetaTheta(optLDA,
                          perc.filt = 0.05,
                          betaScale = 1000)
  deconProp <- results$theta
  deconGexp <- results$beta
  
  dir.create(dir)
  
  for(i in 1:dim(deconProp)[2]){
    plot_data<-merge(pos,deconProp[,i],by = 0)
    names(plot_data)<-c("barcode","x","y","prop")
    
    plot_spatial(plot_data=plot_data,suffix1='_prop.eps',suffix2='_prop.jpg',y_max=1,dir=dir,i=i)
    
  }
    marker_gene_output<-map(.x = 1:dim(deconGexp)[1],
                          ~marker_gene_list(topic = .x,exp_value = 2,Gexp = deconGexp))
  names(marker_gene_output)<-str_c("topic_genes_exp2.",1:dim(deconGexp)[1],".csv")
  
  map2(.x = names(marker_gene_output),
       .y = marker_gene_output,
       ~write_csv(x = .y,file = paste(dir,.x)))
  
  alpha<-map(.x = 1:21,~ldas$model[[.x]]@alpha)%>%unlist
  
  plot_df<-tibble(K=2:22,alpha=alpha,perplexities=ldas$perplexities,rare=ldas$numRare)
  plot_df
  
  p1<-ggplot(data = plot_df) +
    geom_line(mapping = aes(x = K,y = perplexities), color="red3",size=2) +
    geom_point(mapping = aes(x = K,y = perplexities),shape=21, color="black", fill=ifelse(alpha > 1, "white", "red3"), size=6)+
    theme_linedraw(base_size = 16,base_rect_size =2,base_line_size = 2)+ylab("perplexity")+
    ylim(min(plot_df$perplexities)-10,10+max(plot_df$perplexities))
  p2<-ggplot(data = plot_df) +
    geom_point(mapping = aes(x = K,y = rare),shape=21, color="black", fill="blue", size=4)+ 
    geom_line(mapping = aes(x = K,y = rare), color="blue",size=2)+
    theme_linedraw(base_size = 16,base_rect_size =2,base_line_size = 2)+ylab("cell−types with mean proportion < 5%")
  
  p3<-ggplot(data = plot_df) +
    geom_line(mapping = aes(x = K,y = alpha), color="darkgreen",size=2) +ylim(c(0,1))+
    geom_point(mapping = aes(x = K,y = alpha),shape=21, color="black", fill=ifelse(alpha > 1, "white", "darkgreen"), size=6)+
    theme_linedraw(base_size = 16,base_rect_size =2,base_line_size = 2)+ylab("alpha")
  
  print(p1+p2+p3)
  
  ggsave(plot = p1+p2+p3,filename =str_c(dir,"merged_QC_plot.eps") ,height=5, width=12, units='in', dpi=300)
  ggsave(plot = p1+p2+p3,filename = str_c(dir,"merged_QC_plot.jpg"),height=5, width=12, units='in', dpi=300)
  
}

# Load and preprocess data

In [ ]:
counts<-Read10X_h5(filename = "/content/VisiumFFPE_Mouse_Brain_Transgenic_Age_17p9_Rep_1.h5")
counts

In [ ]:
spatial_barcodes<-read_csv("/content/spatial_cord_subset_17p9_rep1.csv")

colnames(counts)%in%spatial_barcodes$barcode%>%summary

In [ ]:
counts_subset <- counts[,colnames(counts)%in%spatial_barcodes$barcode]

pos<-as.data.frame(spatial_barcodes)
rownames(pos)<-pos[,1]

In [ ]:
pos<-pos[,5:6]
names(pos)<-c("x","y")
head(pos)

In [ ]:
counts_subset_clean <- cleanCounts(counts = counts_subset,
                                   min.lib.size = 100,
                                   min.reads = 1,
                                   min.detected = 1,
                                   verbose = TRUE)
## feature select for genes
##I am going to use the over dispersed genes and an astrocyte subset.
odGenes <- getOverdispersedGenes(as.matrix(counts_subset_clean),
                                 gam.k=5,
                                 alpha=0.05,
                                 plot=FALSE,
                                 use.unadjusted.pvals=FALSE,
                                 do.par=TRUE,
                                 max.adjusted.variance=1e3,
                                 min.adjusted.variance=1e-3,
                                 verbose=FALSE, details=TRUE)

In [ ]:
genes <- odGenes$ods
length(genes)

In [ ]:
astro<-read_csv(file = "/content/spatial_cord_subset_17p9_rep1.csv")
astro_overlap_fit<-rownames(counts_subset_clean)%in%astro$`Astrocyte Markers`
astro_overlap<-rownames(counts_subset_clean)[astro_overlap_fit]
gene_astro<-c(genes,astro_overlap)%>%unique()

In [ ]:
corpus<-preprocess(t(as.matrix(counts_subset_clean)),
                   selected.genes = gene_astro,plot=FALSE,
                   min.reads = 1, 
                   min.lib.size = 100, 
                   min.detected = 1,
                   ODgenes = FALSE, 
                   verbose = TRUE)

# LDA model fitting

Find the optimal number of cell-types K for the LDA model.
https://rdrr.io/github/JEFworks-Lab/STdeconvolve/man/fitLDA.html

STdeconvolve: https://jef.works/STdeconvolve/

This can take ~30-60 min:

In [ ]:
ldas <- fitLDA(corpus$corpus, Ks = seq(2, 22, by = 1),
               perc.rare.thresh = 0.05,
               plot=FALSE,
               ncores=4,
               verbose=TRUE)

In [ ]:
saveRDS(object = ldas,file = "/content/optlDA.17p9_rep1_astrogenes.rds")
#ldas<-readRDS(file = "optlDA.17p9_rep1_astrogenes.rds")

In [ ]:
run_me_results(opt=18,dir = "output_18_Tran_17p9_rep1_astrogenes_astro_newplots/")
#run_me_results(opt=12,dir = "output_12_Tran_17p9_rep1_astrogenes_astro/")

This will product numerous warning messages, which can be ignored. 